In [1]:
"""

MONTHLY DATA NOW

"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import yfinance as yf
import fredapi as fa
from datetime import date, timedelta
from Sector_Exposure import Rate_Exposures
import numpy as np
import matplotlib.pyplot as plt
import Useful_Functions as u
from statsmodels.tsa.stattools import adfuller


# sectors_and_rates
cutoff = .7

#factor_df = pd.read_csv("2006.csv", index_col="Date")

# .drop(columns = ["MOMENTUM"])
factor_df = pd.read_csv("regular_factors_and_rates.csv", index_col = "Date").drop(columns = ["MOMENTUM"])

train_df = factor_df[:int(len(factor_df)*cutoff)]
test_df = factor_df[int(len(factor_df)*cutoff):]

forward = 30

exposure = Rate_Exposures(train_df, 252, forward, ['Real Yield', 'Yield Curve'])
for_scores = Rate_Exposures(factor_df, 252, forward, ['Real Yield', 'Yield Curve'])

ry_scores_test_monthly = for_scores.monthly_scores['Real Yield']
yc_scores_test_monthly = for_scores.monthly_scores['Yield Curve']
ry_scores_test_monthly = ry_scores_test_monthly.loc[ ry_scores_test_monthly.index > train_df.index[-1]]
yc_scores_test_monthly = yc_scores_test_monthly.loc[ yc_scores_test_monthly.index > train_df.index[-1]]

train_results_df = exposure.mean_returns_monthly.dropna().copy()
update_df = exposure.forward_returns_monthly.dropna().copy()


In [2]:


update_df



,DEFAULT,QUALITY,BOOK_TO_PRICE,SIZE,CASH_TO_ASSETS,Real Yield_scores,Yield Curve_scores
2011-02,-0.012956,-0.025167,-0.011394,-0.004244,-0.018094,1.0,1.0
2011-03,-0.029073,0.028695,-0.024341,-0.021877,0.022182,-1.0,0.0
2011-04,-0.030605,0.056600,-0.014305,-0.039374,-0.005079,-1.0,1.0
2011-05,-0.024243,0.028795,0.002148,0.000486,-0.024786,-1.0,0.0
2011-06,-0.046067,0.063286,0.027805,0.018814,-0.038357,-1.0,0.0
...,...,...,...,...,...,...,...
2018-01,-0.034657,0.015324,-0.013438,-0.013700,0.078443,1.0,-1.0
2018-02,-0.003106,0.015600,-0.009950,0.022050,0.006456,4.0,0.0
2018-03,0.044945,-0.020910,0.010650,0.010425,-0.047310,2.0,-2.0
2018-04,0.061545,-0.042620,-0.010585,0.026975,0.027290,2.0,-1.0


In [3]:

def closest_combos(combo, in_index = False):

    assert isinstance(combo, tuple)
    
    min_error = 100
    errors = {}

    for rates in train_results_df.index:
        
        ry_error = abs(combo[0] - rates[0])
        yc_error = abs(combo[1] - rates[1])
        error = ry_error + yc_error
        
        
        errors[rates] = error
        
        if error <= min_error and error > 0:
            min_error = error
    
    mins = list(filter(lambda x: errors[x]==min_error, errors))
    
    if in_index:
        mins.append(combo)
    
    return mins


ok = closest_combos((1,1), True)

In [4]:

predicted_best_sector = []
exp_ret = []
scores = []
top_n_sectors = 2

for num, (ry, yc) in enumerate(zip(ry_scores_test_monthly, ry_scores_test_monthly)):
    
    if (ry, yc) in train_results_df.index:
        combos = closest_combos((ry, yc), True)
    else:
        combos = closest_combos((ry, yc))
        
    top_secs = train_results_df.loc[combos].mean().sort_values()[-top_n_sectors:]
        
    predicted_best_sector.append(top_secs.index.to_list())
    exp_ret.append(top_secs.mean())
    scores.append((ry,yc))
    
    

In [5]:

big_df = pd.DataFrame(index = ry_scores_test_monthly.index)
big_df['scores'] = scores
big_df['exp_rets_predicted'] = exp_ret
big_df['pred_top_secs'] = predicted_best_sector


In [6]:


actual_results_exposure = Rate_Exposures(test_df, 252, forward, ['Real Yield', 'Yield Curve'])
actual_returns = actual_results_exposure.forward_returns_monthly.iloc[:,:-2]
actual_returns = actual_returns.loc[ (actual_returns.index >= big_df.index[0]) & (actual_returns.index <= big_df.index[-1])]



In [7]:

actual_secs = []
actual_ret = []
predicted_sector_performance = []
correct_sector_performance = []

for d in actual_returns.index:

    top_actual_secs = actual_returns.loc[str(d)].sort_values()
    actual_secs.append(top_actual_secs.index[-top_n_sectors:].to_list())
    
    actual_ret.append(top_actual_secs[-top_n_sectors:].mean())
    predicted_sector_performance.append(actual_returns.loc[str(d)][list(big_df['pred_top_secs'].loc[str(d)])].mean())

actual_secs = pd.Series(actual_secs, index = actual_returns.index)
actual_ret = pd.Series(actual_ret, index = actual_returns.index)

big_df['actual_top_secs'] = actual_secs
big_df['actual_rets'] = actual_ret

big_df = big_df.dropna()


In [8]:

correct_sectors = []
mean_squared_error_return = []
return_error_regular = []


for pred, act in zip(big_df['pred_top_secs'], big_df['actual_top_secs']):
    sect = set(pred).intersection(set(act))
    correct_sectors.append(len(sect))
    

for pred, act in zip(big_df['exp_rets_predicted'], big_df['actual_rets']):
    
    mean_squared_error_return.append((pred-act)**2)
    return_error_regular.append(pred-act) 
    
big_df['correctly_chosen_sectors'] = correct_sectors
#big_df['correctly_chosen_sectors_above_market'] = correct_sectors_above_market
big_df['predicted_sector_performance'] = predicted_sector_performance
#big_df['correct_sector_performance'] = actual_results['actual_rets']
big_df['mean_squared_error_return_prediction'] = mean_squared_error_return
big_df['return_error_regular'] = return_error_regular

big_df = big_df.dropna()

In [12]:


#big_df.predicted_sector_performance.loc[big_df.correctly_chosen_sectors >= 0].hist()

#big_df.predicted_sector_performance.hist()
#big_df.correctly_chosen_sectors.plot()

big_df



,scores,exp_rets_predicted,pred_top_secs,actual_top_secs,actual_rets,correctly_chosen_sectors,predicted_sector_performance,mean_squared_error_return_prediction,return_error_regular
2018-08,"(0.0, 0.0)",0.009981,"[BOOK_TO_PRICE, QUALITY]","[BOOK_TO_PRICE, DEFAULT]",0.004200,1,-0.000350,0.000033,0.005781
2018-09,"(2.0, 2.0)",0.028530,"[QUALITY, CASH_TO_ASSETS]","[BOOK_TO_PRICE, QUALITY]",0.026711,1,0.003995,0.000003,0.001819
2018-10,"(3.0, 3.0)",0.033948,"[SIZE, CASH_TO_ASSETS]","[CASH_TO_ASSETS, QUALITY]",0.051479,1,-0.019045,0.000307,-0.017531
2018-11,"(2.0, 2.0)",0.028530,"[QUALITY, CASH_TO_ASSETS]","[CASH_TO_ASSETS, QUALITY]",0.057108,2,0.057108,0.000817,-0.028578
2018-12,"(0.0, 0.0)",0.009981,"[BOOK_TO_PRICE, QUALITY]","[CASH_TO_ASSETS, DEFAULT]",0.059063,0,-0.015982,0.002409,-0.049082
2019-01,"(-0.0, -0.0)",0.009981,"[BOOK_TO_PRICE, QUALITY]","[SIZE, CASH_TO_ASSETS]",0.027250,0,-0.020438,0.000298,-0.017269
2019-02,"(-2.0, -2.0)",0.022746,"[CASH_TO_ASSETS, DEFAULT]","[QUALITY, CASH_TO_ASSETS]",0.010233,1,-0.017947,0.000157,0.012512
2019-03,"(-3.0, -3.0)",0.045978,"[CASH_TO_ASSETS, DEFAULT]","[CASH_TO_ASSETS, QUALITY]",0.006273,1,-0.014355,0.001577,0.039706
2019-04,"(-2.0, -2.0)",0.022746,"[CASH_TO_ASSETS, DEFAULT]","[BOOK_TO_PRICE, QUALITY]",0.010933,0,-0.054270,0.000140,0.011813
2019-05,"(-2.0, -2.0)",0.022746,"[CASH_TO_ASSETS, DEFAULT]","[CASH_TO_ASSETS, QUALITY]",0.027186,1,-0.011419,0.000020,-0.004440
